In [25]:
!python -V

Python 3.9.15


In [31]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import mlflow

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [32]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
    numerical = ['trip_distance']
    test_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(test_dicts)

def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}


In [30]:
df_test = read_dataframe('/home/pmspraju/tracking-server/data/green_tripdata_2021-03.parquet')

In [33]:
len(df_test)

80372

In [34]:
target = 'duration'
y_test = df_test[target].values

In [14]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI ="sqlite:////home/pmspraju/tracking-server/mlflow.db" 

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [8]:
# List experiments
experiments = client.search_experiments()
for ex in experiments:
    print(ex._name)

nyc-taxi-experiment-1


In [13]:
# search runs
from mlflow.entities import ViewType
runs = client.search_runs(
    experiment_ids='2',
    filter_string="metrics.rmse < 6.5",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

for run in runs:
    print(f"runid: {run.info.run_id}, rmse: {run.data.metrics['rmse']}.4f, mlflow_name: {run.info.run_name}")

runid: bbd69a41e3df45d3b56eea857ecd0721, rmse: 6.4416226198779905.4f, mlflow_name: sassy-asp-693
runid: 5a758a6015de423c9b55f8b820654939, rmse: 6.458027230161294.4f, mlflow_name: sassy-moose-331
runid: 9d8bdf139bce47f88a2a61029470e909, rmse: 6.4598527549080185.4f, mlflow_name: silent-sponge-146


In [18]:
# List registered models
reg_models = client.search_registered_models()
for m in reg_models:
    print(m.name)

nyc-taxi-regressor


In [20]:
# Register a version to an existing model
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
run_id = "bbd69a41e3df45d3b56eea857ecd0721"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/12/24 08:22:06 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1671891726137, current_stage='None', description=None, last_updated_timestamp=1671891726137, name='nyc-taxi-regressor', run_id='bbd69a41e3df45d3b56eea857ecd0721', run_link=None, source='//home/pmspraju/tracking-server/mlruns/bbd69a41e3df45d3b56eea857ecd0721/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [21]:
# list of the model versions
model_name = "nyc-taxi-regressor"
versions = client.get_latest_versions(name=model_name)
for version in versions:
    print(f"version:{version.version},current stage:{version.current_stage}")

version:2,current stage:Staging
version:3,current stage:None


In [22]:
# Transition a version 
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Staging",
    archive_existing_versions=False

)

<ModelVersion: creation_timestamp=1671891726137, current_stage='Staging', description=None, last_updated_timestamp=1671892216766, name='nyc-taxi-regressor', run_id='bbd69a41e3df45d3b56eea857ecd0721', run_link=None, source='//home/pmspraju/tracking-server/mlruns/bbd69a41e3df45d3b56eea857ecd0721/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [24]:
# Update a version attribute like description
from datetime import datetime
date= datetime.today().date()
client.update_model_version(
    name=model_name,
    version=3,
    description=f"This model version was updated on {date}"
)

<ModelVersion: creation_timestamp=1671891726137, current_stage='Staging', description='This model version was updated on 2022-12-24', last_updated_timestamp=1671892546969, name='nyc-taxi-regressor', run_id='bbd69a41e3df45d3b56eea857ecd0721', run_link=None, source='//home/pmspraju/tracking-server/mlruns/bbd69a41e3df45d3b56eea857ecd0721/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [35]:
run_id = "9d8bdf139bce47f88a2a61029470e909"
model_uri = f"runs:/{run_id}/model"
client.download_artifacts(
    run_id=run_id,
    path='preprocessor',
    dst_path='/home/pmspraju/tracking-server/test-models'
)

/tmp/ipykernel_987/2655490884.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(


'/home/pmspraju/tracking-server/test-models/preprocessor'

In [36]:
with open("/home/pmspraju/tracking-server/test-models/preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [37]:
X_test = preprocess(df_test, dv)

In [40]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

[09:20:20] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 3.35 s, sys: 0 ns, total: 3.35 s
Wall time: 1.62 s


{'rmse': 6.39181859348981}

In [42]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 203 ms, sys: 0 ns, total: 203 ms
Wall time: 199 ms


{'rmse': 6.659623830022514}